In [3]:
import pandas as pd
from io import StringIO
import requests
from tqdm import tqdm
import ast

df=pd.read_csv(r"datasets/01a_Sentence_df.csv", sep="|", lineterminator='\n')
df["taggedSentences"]  = [ast.literal_eval(text) for text in df["taggedSentences"]]
display(df.head())

# Sentence numbers including sentences with only punctuation
# A     7141
# B    20087
# C    19318

df = df[df["Text"] != '.']
df = df[df["Text"] != '!']
df = df[df["Text"] != ':']
df = df[df["Text"] != '?' ]
df = df[df["Text"] != '*']
df = df[df["Text"] != ","]
df = df[df["Text"] != '-']
df = df[df["Text"] != '.,']
df = df[df["Text"] != '. ,']
df = df[df['Text'].apply(lambda x: len(x)>3)]
df = df[df['taggedSentences'].apply(lambda x: len(x)>1)]

df = df[df.newLevel != "L"]
level = df.groupby('newLevel')
print(level.size())
df = level.apply(lambda x: x.sample(level.size().min()).reset_index(drop=True))
print(len(df[df["newLevel"]=="A"] ))

,Text,newLevel,taggedSentences,parsedSentences
0,Zu meiner Familie gehören vier Personen.,A,"[[Zu, APPR, Dat], [meiner, PRO, Poss, Attr, -,...",(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN Fam...
1,Die Mutter bin ich und dann gehört natürlich m...,A,"[[Die, ART, Def, Nom, Sg, Fem], [Mutter, N, Re...",(ROOT (CS (S (NP (ART Die) (NN Mutter)) (VAFIN...
2,"Wir haben zwei Kinder, einen Sohn, der sechs J...",A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [habe...",(ROOT (S (PPER Wir) (VAFIN haben) (CNP (NP (CA...
3,Wir wohnen in einem kleinen Haus mit einem Gar...,A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [wohn...",(ROOT (S (PPER Wir) (VVFIN wohnen) (PP (APPR i...
4,Dort können die Kinder ein bisschen spielen.,A,"[[Dort, ADV], [können, VFIN, Mod, 3, Pl, Pres,...",(ROOT (S (ADV Dort) (VMFIN können) (NP (ART di...


newLevel
A     6898
B    19652
C    19015
dtype: int64
6898


In [5]:
from sklearn.model_selection import train_test_split

def save_df(path,df):
    df.to_csv(path,sep="|",index=False)
    
###split training and test data###
x_train ,x_test = train_test_split(df,test_size=0.3)

for df_inProcessing, df_name in tqdm([(x_train, "train"), (x_test, "test")]):
    for N in tqdm(range(1,16)): # number of sentences per Text
        ###Text with len N sentence###
        rows = []

        complexityLevels = ['A', 'B', 'C']
        
        no_of_text = min([int(len(df_inProcessing[(df["newLevel"] == complexityLevel)])/N) for complexityLevel in complexityLevels])
        print(no_of_text)
        for complexityLevel in complexityLevels:
            
            for n in tqdm(range(no_of_text)): #number of text per level
                df_sample = df_inProcessing[(df["newLevel"] == complexityLevel)].sample(N)
                rows.append([" ".join(df_sample['Text'].tolist()), complexityLevel, df_sample['taggedSentences'].tolist(), df_sample['parsedSentences'].tolist()])
        random_df = pd.DataFrame(rows, columns=['cleanedText', 'newLevel', 'RFTagger', 'parsedText'])

        display(random_df.head())

        filename = str("datasets/01_RawDataset/" + df_name + "_" + str(N) + "_df.csv")
        save_df(filename, random_df)

  0%|          | 0/15 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


  0%|          | 0/4815 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


  0%|          | 14/4815 [00:00<00:34, 138.36it/s]

4815




  1%|          | 29/4815 [00:00<00:34, 140.06it/s]

  1%|          | 43/4815 [00:00<00:34, 138.36it/s]

  1%|          | 58/4815 [00:00<00:33, 140.99it/s]

  2%|▏         | 73/4815 [00:00<00:33, 142.46it/s]

  2%|▏         | 86/4815 [00:00<00:34, 137.20it/s]

  2%|▏         | 102/4815 [00:00<00:33, 139.46it/s]

  2%|▏         | 115/4815 [00:00<00:34, 135.72it/s]

  3%|▎         | 129/4815 [00:00<00:34, 135.75it/s]

  3%|▎         | 142/4815 [00:01<00:34, 134.97it/s]

  3%|▎         | 155/4815 [00:01<00:34, 134.12it/s]

  3%|▎         | 168/4815 [00:01<00:34, 133.62it/s]

  4%|▍         | 182/4815 [00:01<00:34, 133.96it/s]

  4%|▍         | 197/4815 [00:01<00:34, 134.85it/s]

  4%|▍         | 211/4815 [00:01<00:34, 135.13it/s]

  5%|▍         | 225/4815 [00:01<00:34, 134.85it/s]

  5%|▍         | 240/4815 [00:01<00:33, 135.57it/s]

  5%|▌         | 255/4815 [00:01<00:33, 136.07it/s]

  6%|▌         | 271/4815 [00:01<00:33, 136.77it/s]

  6%|▌         | 287/4815 [00:02<00:32, 137.59it/

 97%|█████████▋| 4671/4815 [00:28<00:00, 161.55it/s]

 97%|█████████▋| 4689/4815 [00:29<00:00, 161.60it/s]

 98%|█████████▊| 4707/4815 [00:29<00:00, 161.65it/s]

 98%|█████████▊| 4725/4815 [00:29<00:00, 161.69it/s]

 99%|█████████▊| 4743/4815 [00:29<00:00, 161.74it/s]

 99%|█████████▉| 4761/4815 [00:29<00:00, 161.80it/s]

 99%|█████████▉| 4779/4815 [00:29<00:00, 161.85it/s]

100%|█████████▉| 4797/4815 [00:29<00:00, 161.89it/s]

100%|██████████| 4815/4815 [00:29<00:00, 161.93it/s]



  0%|          | 0/4815 [00:00<?, ?it/s]

  0%|          | 17/4815 [00:00<00:29, 164.38it/s]

  1%|          | 34/4815 [00:00<00:29, 162.66it/s]

  1%|          | 51/4815 [00:00<00:29, 163.75it/s]

  1%|▏         | 68/4815 [00:00<00:29, 163.44it/s]

  2%|▏         | 85/4815 [00:00<00:28, 164.25it/s]

  2%|▏         | 102/4815 [00:00<00:28, 164.59it/s]

  2%|▏         | 119/4815 [00:00<00:28, 164.87it/s]

  3%|▎         | 136/4815 [00:00<00:28, 164.53it/s]

  3%|▎         | 153/4815 [00:00<00:28, 164.20it/s]

 94%|█████████▍| 4550/4815 [00:27<00:01, 166.04it/s]

 95%|█████████▍| 4566/4815 [00:27<00:01, 166.01it/s]

 95%|█████████▌| 4583/4815 [00:27<00:01, 166.01it/s]

 96%|█████████▌| 4600/4815 [00:27<00:01, 166.01it/s]

 96%|█████████▌| 4617/4815 [00:27<00:01, 166.00it/s]

 96%|█████████▌| 4634/4815 [00:27<00:01, 165.99it/s]

 97%|█████████▋| 4651/4815 [00:28<00:00, 165.96it/s]

 97%|█████████▋| 4668/4815 [00:28<00:00, 165.95it/s]

 97%|█████████▋| 4685/4815 [00:28<00:00, 165.94it/s]

 98%|█████████▊| 4702/4815 [00:28<00:00, 165.93it/s]

 98%|█████████▊| 4719/4815 [00:28<00:00, 165.94it/s]

 98%|█████████▊| 4737/4815 [00:28<00:00, 165.96it/s]

 99%|█████████▉| 4755/4815 [00:28<00:00, 165.98it/s]

 99%|█████████▉| 4772/4815 [00:28<00:00, 165.99it/s]

 99%|█████████▉| 4789/4815 [00:28<00:00, 166.00it/s]

100%|█████████▉| 4806/4815 [00:28<00:00, 165.99it/s]

100%|██████████| 4815/4815 [00:29<00:00, 165.96it/s]

  0%|          | 0/4815 [00:00<?, ?it/s]

  0%|          | 17/4815 [00:00<00:29, 1

...


 93%|█████████▎| 14/15 [01:32<00:06,  6.58s/it]

  0%|          | 0/136 [00:00<?, ?it/s]

 18%|█▊        | 24/136 [00:00<00:00, 237.60it/s]

136




 35%|███▍      | 47/136 [00:00<00:00, 233.22it/s]

 52%|█████▏    | 71/136 [00:00<00:00, 233.61it/s]

 70%|██████▉   | 95/136 [00:00<00:00, 234.59it/s]

 85%|████████▌ | 116/136 [00:00<00:00, 229.26it/s]

100%|██████████| 136/136 [00:00<00:00, 231.07it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

 17%|█▋        | 23/136 [00:00<00:00, 218.21it/s]

 33%|███▎      | 45/136 [00:00<00:00, 217.92it/s]

 49%|████▉     | 67/136 [00:00<00:00, 215.98it/s]

 66%|██████▌   | 90/136 [00:00<00:00, 218.83it/s]

 83%|████████▎ | 113/136 [00:00<00:00, 220.08it/s]

100%|██████████| 136/136 [00:00<00:00, 220.43it/s]



  0%|          | 0/136 [00:00<?, ?it/s]

 16%|█▌        | 22/136 [00:00<00:00, 218.66it/s]

 34%|███▍      | 46/136 [00:00<00:00, 227.26it/s]

 51%|█████▏    | 70/136 [00:00<00:00, 229.17it/s]

 69%|██████▉   | 94/136 [00:00<00:00, 230.92it/s]

 86%|████████▌ | 117/136 [00:00<00:00, 230.60it/s]

100%|██████████| 136/136 [00:00<00:00, 230.19it/s]

,cleanedText,newLevel,RFTagger,parsedText
0,Wir brachen einen Barhocker . Etwas silberne F...,A,"[[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [bra...",[(ROOT (S (PPER Wir) (VVFIN brachen) (NP (ART ...
1,"Lieber Ingo, wie geht es Dir? Ich öffnete die ...",A,"[[[Lieber, N, Reg, Nom, Sg, Masc], [Ingo, N, N...",[(ROOT (CS (S (ADJD Lieber) (NP (NE Ingo))) ($...
2,Du bist Jets ein Vater. Hast du Zeit? Kann du ...,A,"[[[Du, PRO, Pers, Subst, 2, Nom, Sg, *], [bist...",[(ROOT (S (PPER Du) (VAFIN bist) (NP (NN Jets)...
3,Ich suche ein Wohnung mit ein Zimmer und mit b...,A,"[[[Ich, PRO, Pers, Subst, 1, Nom, Sg, *], [suc...",[(ROOT (S (PPER Ich) (VVFIN suche) (NP (ART ei...
4,Ich frage meinen Freundin Julia brauchen Etike...,A,"[[[Ich, PRO, Pers, Subst, 1, Nom, Sg, *], [fra...",[(ROOT (S (PPER Ich) (VVFIN frage) (NP (PPOSAT...



100%|██████████| 2/2 [06:28<00:00, 194.39s/it]
